In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:1]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fruit-and-vegetable-image-recognition/validation/capsicum/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/sweetcorn/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/orange/Image_1.png
/kaggle/input/fruit-and-vegetable-image-recognition/validation/tomato/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/turnip/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/ginger/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/raddish/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/pomegranate/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/pineapple/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/jalepeno/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/apple/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/carrot/Image_2.jpg
/

In [2]:
!pip -q install git+https://github.com/PerforatedAI/PerforatedAI.git

  Preparing metadata (setup.py) ... done


In [7]:
from __future__ import print_function

import argparse
import json
import os
import random
import time

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights

# PerforatedAI 
from perforatedai import globals_perforatedai as GPA
from perforatedai import utils_perforatedai as UPA


# ────────────────────────────────────────────────────────────────
# Utils
# ────────────────────────────────────────────────────────────────
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def ensure_dir(path: str):
    if path and not os.path.exists(path):
        os.makedirs(path, exist_ok=True)


def count_params(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# ────────────────────────────────────────────────────────────────
# Model
# ────────────────────────────────────────────────────────────────
def build_model(num_classes: int):
    weights = MobileNet_V3_Small_Weights.DEFAULT
    model = mobilenet_v3_small(weights=weights)
    in_f = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(in_f, num_classes)
    return model, weights


# ────────────────────────────────────────────────────────────────
# PerforatedAI config GD dendrites only
# ────────────────────────────────────────────────────────────────
def configure_perforatedai(args):
    """
    Path A compliance:
      - uses initialize_pai()
      - uses add_validation_score()
      - generates PAI/PAI.png
    And critically:
      - NO Perforated Backprop (PB), so no UPB/initialize_pb required.
    """

    # Improvement threshold schedule
    if args.improvement_threshold == 0:
        thresh = [0.01, 0.001, 0.0001, 0]
    elif args.improvement_threshold == 1:
        thresh = [0.001, 0.0001, 0]
    else:
        thresh = [0]
    GPA.pc.set_improvement_threshold(thresh)

    GPA.pc.set_candidate_weight_initialization_multiplier(
        args.candidate_weight_init_mult
    )

    # Forward function used inside dendrites
    if args.pai_forward_function == "sigmoid":
        pai_forward = torch.sigmoid
    elif args.pai_forward_function == "relu":
        pai_forward = torch.relu
    elif args.pai_forward_function == "tanh":
        pai_forward = torch.tanh
    else:
        pai_forward = torch.sigmoid
    GPA.pc.set_pai_forward_function(pai_forward)

    # PATH A (GD dendrites):
    # IMPORTANT: keep PB OFF to avoid any UPB/initialize_pb dependency.
    if args.dendrite_mode == 0:
        GPA.pc.set_max_dendrites(0)
    else:
        GPA.pc.set_max_dendrites(int(args.max_dendrites))

    GPA.pc.set_perforated_backpropagation(False)  

    # Recommended defaults
    GPA.pc.set_testing_dendrite_capacity(False)
    GPA.pc.set_weight_decay_accepted(True)
    GPA.pc.set_verbose(False)


# ────────────────────────────────────────────────────────────────
# Train / Eval (PerforatedAI-aware)
# ────────────────────────────────────────────────────────────────
def train_one_epoch(args, model, device, train_loader, optimizer, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        output = model(data)
        loss = nn.functional.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        bs = data.size(0)
        running_loss += loss.item() * bs
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += bs

        if batch_idx % args.log_interval == 0:
            pct = 100.0 * batch_idx / max(1, len(train_loader))
            acc = 100.0 * correct / max(1, total)
            print(
                f"Train Epoch: {epoch} [{batch_idx*bs}/{len(train_loader.dataset)} ({pct:.0f}%)] "
                f"Loss: {loss.item():.6f}  Acc: {acc:.2f}%"
            )
            if args.dry_run:
                break

    train_loss = running_loss / max(1, total)
    train_acc = 100.0 * correct / max(1, total)

    # Feed PerforatedAI the train score
    GPA.pai_tracker.add_extra_score(train_acc, "train")

    model.to(device)
    return train_loss, train_acc


@torch.no_grad()
def evaluate_and_maybe_restructure(args, model, device, loader, optimizer, scheduler, split_name="validation"):
    model.eval()
    loss_sum = 0.0
    correct = 0
    total = 0

    for data, target in loader:
        data = data.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)

        output = model(data)
        loss = nn.functional.cross_entropy(output, target, reduction="sum")
        loss_sum += loss.item()

        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)

    avg_loss = loss_sum / max(1, total)
    acc = 100.0 * correct / max(1, total)

    print(
        f"\n{split_name.capitalize()} set: Average loss: {avg_loss:.4f}, "
        f"Accuracy: {correct}/{total} ({acc:.2f}%)\n"
    )

    #  PerforatedAI validation hook 
    model, restructured, training_complete = GPA.pai_tracker.add_validation_score(acc, model)
    model.to(device)

    # If model changed, reset optimizer/scheduler via tracker
    if restructured and not training_complete:
        optim_args = {"params": model.parameters(), "lr": args.lr, "weight_decay": args.weight_decay}
        sched_args = {"step_size": 1, "gamma": args.gamma}
        optimizer, scheduler = GPA.pai_tracker.setup_optimizer(model, optim_args, sched_args)
        print("[INFO] Model restructured → optimizer/scheduler reset.")

    return avg_loss, acc, model, optimizer, scheduler, training_complete, restructured


# ────────────────────────────────────────────────────────────────
# Main
# ────────────────────────────────────────────────────────────────
def main():
    parser = argparse.ArgumentParser(
        description="Fruit/Veg — PerforatedAI  (GD dendrites) on MobileNetV3-Small"
    )

    # Kaggle dataset dirs 
    parser.add_argument("--train-dir", type=str, default="/kaggle/input/fruit-and-vegetable-image-recognition/train")
    parser.add_argument("--val-dir", type=str, default="/kaggle/input/fruit-and-vegetable-image-recognition/validation")
    parser.add_argument("--test-dir", type=str, default="/kaggle/input/fruit-and-vegetable-image-recognition/test")

    # Training hyperparams
    parser.add_argument("--batch-size", type=int, default=64)
    parser.add_argument("--test-batch-size", type=int, default=128)
    parser.add_argument("--epochs", type=int, default=200)  # PAI may stop early
    parser.add_argument("--lr", type=float, default=3e-4)
    parser.add_argument("--gamma", type=float, default=0.9)
    parser.add_argument("--weight-decay", type=float, default=1e-4)

    # Runtime
    parser.add_argument("--no-cuda", action="store_true", default=False)
    parser.add_argument("--dry-run", action="store_true", default=False)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--log-interval", type=int, default=20)
    parser.add_argument("--num-workers", type=int, default=2)

    # Outputs
    parser.add_argument("--save-model", action="store_true", default=True)
    parser.add_argument("--out-dir", type=str, default="./outputs")
    parser.add_argument("--model-name", type=str, default="fruitveg_mnv3s_pathA_gd_dendritic.pt")
    parser.add_argument("--metrics-name", type=str, default="pathA_gd_dendritic_metrics.json")
    parser.add_argument("--save-name", type=str, default="PAI")  # this yields ./PAI/PAI.png

    # PerforatedAI knobs (Path A)
    parser.add_argument("--dendrite-mode", type=int, default=1, choices=[0, 1])  # 0=control, 1=GD dendrites
    parser.add_argument("--max-dendrites", type=int, default=5)
    parser.add_argument("--improvement-threshold", type=int, default=1, choices=[0, 1, 2])
    parser.add_argument("--candidate-weight-init-mult", type=float, default=0.01)
    parser.add_argument("--pai-forward-function", type=str, default="sigmoid", choices=["sigmoid", "relu", "tanh"])

    args = parser.parse_args()

    # Validate dirs
    for p in [args.train_dir, args.val_dir, args.test_dir]:
        if not os.path.isdir(p):
            raise FileNotFoundError(f"Directory not found: {p}")

    use_cuda = (not args.no_cuda) and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print("[INFO] device:", device)

    set_seed(args.seed)
    ensure_dir(args.out_dir)

    # Transforms: keep aligned to your baseline for fair comparison
    _, weights = build_model(num_classes=2)

    IMAGENET_MEAN = (0.485, 0.456, 0.406)
    IMAGENET_STD = (0.229, 0.224, 0.225)

    train_transform = transforms.Compose(
        [
            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.10),
            transforms.ToTensor(),
            transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
        ]
    )
    eval_transform = weights.transforms()

    train_ds = datasets.ImageFolder(args.train_dir, transform=train_transform)
    val_ds = datasets.ImageFolder(args.val_dir, transform=eval_transform)
    test_ds = datasets.ImageFolder(args.test_dir, transform=eval_transform)

    # Ensure mapping consistency across splits
    if val_ds.class_to_idx != train_ds.class_to_idx:
        raise ValueError(
            "Class mapping mismatch train vs val.\n"
            f"train: {train_ds.class_to_idx}\nval: {val_ds.class_to_idx}"
        )
    if test_ds.class_to_idx != train_ds.class_to_idx:
        raise ValueError(
            "Class mapping mismatch train vs test.\n"
            f"train: {train_ds.class_to_idx}\ntest: {test_ds.class_to_idx}"
        )

    num_classes = len(train_ds.classes)
    print("[INFO] num_classes:", num_classes)
    print("[INFO] example classes:", train_ds.classes[:10], "...")

    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=args.num_workers,
        pin_memory=use_cuda,
    )
    val_loader = torch.utils.data.DataLoader(
        val_ds,
        batch_size=args.test_batch_size,
        shuffle=False,
        num_workers=args.num_workers,
        pin_memory=use_cuda,
    )
    test_loader = torch.utils.data.DataLoader(
        test_ds,
        batch_size=args.test_batch_size,
        shuffle=False,
        num_workers=args.num_workers,
        pin_memory=use_cuda,
    )

    # Build base model
    model, _ = build_model(num_classes=num_classes)
    model = model.to(device)

    base_params = count_params(model)
    print(f"[INFO] base trainable params: {base_params:,} ({base_params/1e6:.2f}M)")

    # Configure PerforatedAI 
    configure_perforatedai(args)

    

    # Avoid interactive pdb stop in Kaggle
    GPA.pc.set_unwrapped_modules_confirmed(True)
    GPA.pc.set_verbose(False)


    #Mandatory hook: initialize_pai wraps the model + creates graph artifacts
    model = UPA.initialize_pai(model, save_name=args.save_name)
    model.to(device)

    # Setup optimizer / scheduler THROUGH PAI tracker (important)
    GPA.pai_tracker.set_optimizer(optim.AdamW)
    GPA.pai_tracker.set_scheduler(StepLR)

    optim_args = {"params": model.parameters(), "lr": args.lr, "weight_decay": args.weight_decay}
    sched_args = {"step_size": 1, "gamma": args.gamma}
    optimizer, scheduler = GPA.pai_tracker.setup_optimizer(model, optim_args, sched_args)

    best_val_acc = -1.0
    best_state = None
    history = []
    start_time = time.time()

    training_complete = False

    for epoch in range(1, args.epochs + 1):
        tr_loss, tr_acc = train_one_epoch(args, model, device, train_loader, optimizer, epoch)

        val_loss, val_acc, model, optimizer, scheduler, training_complete, restructured = (
            evaluate_and_maybe_restructure(args, model, device, val_loader, optimizer, scheduler, "validation")
        )

        scheduler.step()

        num_dendrites = int(GPA.pai_tracker.member_vars.get("num_dendrites_added", 0))
        mode = str(GPA.pai_tracker.member_vars.get("mode", ""))

        history.append(
            {
                "epoch": epoch,
                "train_loss": tr_loss,
                "train_acc": tr_acc,
                "val_loss": val_loss,
                "val_acc": val_acc,
                "lr": optimizer.param_groups[0]["lr"],
                "pai_num_dendrites_added": num_dendrites,
                "pai_mode": mode,
                "restructured": bool(restructured),
            }
        )

        print(f"[INFO] PAI mode={mode} | dendrites_added={num_dendrites} | restructured={restructured}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = {
                "model_state": model.state_dict(),
                "classes": train_ds.classes,
                "class_to_idx": train_ds.class_to_idx,
                "best_val_acc": best_val_acc,
                "base_params": base_params,
                "args": vars(args),
            }
            if args.save_model:
                out_path = os.path.join(args.out_dir, args.model_name)
                torch.save(best_state, out_path)
                print(f"[INFO] Saved best model to {out_path}")

        if args.dry_run:
            break

        if training_complete:
            print("[INFO] PerforatedAI reports training complete. Stopping early.")
            break

    # Load best checkpoint for final test
    if best_state is not None and args.save_model:
        ckpt_path = os.path.join(args.out_dir, args.model_name)
        ckpt = torch.load(ckpt_path, map_location=device)
        model.load_state_dict(ckpt["model_state"])

    # Final test evaluation (no restructuring)
    model.eval()
    test_loss_sum = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device, non_blocking=True)
            target = target.to(device, non_blocking=True)
            output = model(data)
            loss = nn.functional.cross_entropy(output, target, reduction="sum")
            test_loss_sum += loss.item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)

    test_loss = test_loss_sum / max(1, total)
    test_acc = 100.0 * correct / max(1, total)

    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{total} ({test_acc:.2f}%)\n")

    elapsed = time.time() - start_time
    num_dendrites_final = int(GPA.pai_tracker.member_vars.get("num_dendrites_added", 0))
    mode_final = str(GPA.pai_tracker.member_vars.get("mode", ""))

    metrics = {
        "best_val_acc": best_val_acc,
        "final_test_acc": test_acc,
        "final_test_loss": test_loss,
        "base_params": base_params,
        "elapsed_seconds": elapsed,
        "pai_num_dendrites_added": num_dendrites_final,
        "pai_mode": mode_final,
        "history": history,
        "required_graph_path": f"./{args.save_name}/{args.save_name}.png",  # typically ./PAI/PAI.png
    }

    metrics_path = os.path.join(args.out_dir, args.metrics_name)
    with open(metrics_path, "w") as f:
        json.dump(metrics, f, indent=2)

    print(f"[DONE] Metrics saved to: {metrics_path}")
    print(f"[DONE] Best Val Acc: {best_val_acc:.2f}% | Test Acc: {test_acc:.2f}%")
    print(f"[DONE] Time: {elapsed/60:.1f} minutes")

    print("\n[IMPORTANT] For judging, include the PerforatedAI graph image:")
    print(f"  -> ./{args.save_name}/{args.save_name}.png  (usually ./PAI/PAI.png)\n")


if __name__ == "__main__":
    import sys
    # Kaggle notebooks inject args like `-f ...json` → wipe them so argparse doesn't crash
    sys.argv = [""]
    main()

[INFO] device: cuda
[INFO] num_classes: 36
[INFO] example classes: ['apple', 'banana', 'beetroot', 'bell pepper', 'cabbage', 'capsicum', 'carrot', 'cauliflower', 'chilli pepper', 'corn'] ...
[INFO] base trainable params: 1,554,756 (1.55M)
Running Dendrite Experiment
Train Epoch: 1 [0/3115 (0%)] Loss: 3.693619  Acc: 1.56%


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Epoch: 1 [1280/3115 (41%)] Loss: 2.609549  Acc: 25.60%
Train Epoch: 1 [2560/3115 (82%)] Loss: 1.402737  Acc: 40.17%

Validation set: Average loss: 0.8475, Accuracy: 262/351 (74.64%)

Adding validation score 74.64387464
Checking PAI switch with mode n, switch mode DOING_HISTORY, epoch 0, last improved epoch 0, total epochs 0, n: 10, num_cycles: 0
Returning False - no triggers to switch have been hit
[INFO] PAI mode=n | dendrites_added=0 | restructured=0
[INFO] Saved best model to ./outputs/fruitveg_mnv3s_pathA_gd_dendritic.pt
Train Epoch: 2 [0/3115 (0%)] Loss: 0.886523  Acc: 79.69%


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Epoch: 2 [1280/3115 (41%)] Loss: 0.818896  Acc: 75.89%
Train Epoch: 2 [2560/3115 (82%)] Loss: 0.672635  Acc: 77.02%

Validation set: Average loss: 0.4083, Accuracy: 301/351 (85.75%)

Adding validation score 85.75498575
Checking PAI switch with mode n, switch mode DOING_HISTORY, epoch 1, last improved epoch 1, total epochs 1, n: 10, num_cycles: 0
Returning False - no triggers to switch have been hit
[INFO] PAI mode=n | dendrites_added=0 | restructured=0
[INFO] Saved best model to ./outputs/fruitveg_mnv3s_pathA_gd_dendritic.pt
Train Epoch: 3 [0/3115 (0%)] Loss: 0.431152  Acc: 87.50%


...

Train Epoch: 143 [1280/3115 (41%)] Loss: 0.001147  Acc: 99.48%
Train Epoch: 143 [2560/3115 (82%)] Loss: 0.010646  Acc: 99.50%

Validation set: Average loss: 0.1446, Accuracy: 338/351 (96.30%)

Adding validation score 96.29629630
Checking PAI switch with mode n, switch mode DOING_HISTORY, epoch 65, last improved epoch 57, total epochs 134, n: 10, num_cycles: 10
Returning False - no triggers to switch have been hit
[INFO] PAI mode=n | dendrites_added=5 | restructured=0
Train Epoch: 144 [0/3115 (0%)] Loss: 0.000860  Acc: 100.00%


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Epoch: 144 [1280/3115 (41%)] Loss: 0.001625  Acc: 99.11%
Train Epoch: 144 [2560/3115 (82%)] Loss: 0.012424  Acc: 98.97%

Validation set: Average loss: 0.1456, Accuracy: 337/351 (96.01%)

Adding validation score 96.01139601
Checking PAI switch with mode n, switch mode DOING_HISTORY, epoch 66, last improved epoch 57, total epochs 135, n: 10, num_cycles: 10
Returning False - no triggers to switch have been hit
[INFO] PAI mode=n | dendrites_added=5 | restructured=0


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Epoch: 145 [0/3115 (0%)] Loss: 0.004826  Acc: 100.00%
Train Epoch: 145 [1280/3115 (41%)] Loss: 0.003070  Acc: 99.40%
Train Epoch: 145 [2560/3115 (82%)] Loss: 0.000655  Acc: 99.31%

Validation set: Average loss: 0.1463, Accuracy: 338/351 (96.30%)

Adding validation score 96.29629630
Checking PAI switch with mode n, switch mode DOING_HISTORY, epoch 67, last improved epoch 57, total epochs 136, n: 10, num_cycles: 10
Returning True - History and last improved is hit
Dendrites did not improve system and 2 > 2 so returning training_complete.
You should now exit your training loop and best_model will be your final model for inference
For improved results, try perforated backpropagation next time!
before graphs
after graphs
after save
[INFO] PAI mode=n | dendrites_added=4 | restructured=True
[INFO] PerforatedAI reports training complete. Stopping early.

Test set: Average loss: 0.1184, Accuracy: 350/359 (97.49%)

[DONE] Metrics saved to: ./outputs/pathA_gd_dendritic_metrics.json
[DONE] B